In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, mmread
import sys
import itertools 
import pickle

In [2]:
import rankingmethod
import rankingmeasure

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from experiment import *

##Epinion

In [10]:
data = pd.read_csv("../datasets/soc-Epinions1.txt", sep='\t', header=4)
data = data.as_matrix()

In [ ]:
#создание epinion удалив от туда пользователей, у которых количество отзывов меньше 25
#Также проводится переиндексанция пользователей и предметов

In [11]:
epinion, user_item = make_valid_data(data)

In [12]:
train, test, traintest = givenK_train_test(epinion, 5)

In [13]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = poprec.get_list(0)[:3]
# all_measures(train, test, poprec, True, top)
all_measures(train, test, poprec, True)

precision5 0.194733257662
1-call5 0.5675368898978433
MRR 0.44533300948174925
AUC 0.830667376404
NGDC 0.219722294304


In [ ]:
###CLimf подбор параметров(не используя фичу авторов)

In [ ]:
list_reg = [0.001, 0.05, 0.01, 0.005 , 0.001]
list_lrate = [0.01, 0.0001, 0.005, 0.01, 0.05, 0.1 ]
list_K = [10, 5, 7, 12, 15]

climf_validation = np.zeros((len(list_reg), len(list_lrate), len(list_K)))

for i, K in enumerate(list_K):
    for j, lrate in enumerate(list_lrate):
        for m, reg in enumerate(list_reg):
            climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=10)
            climf.fit(train)
            mrr = rankingmeasure.get_MRR(train, test, 
            climf, skip_train=False)
            climf_validation[i,j,m] = mrr
            print(K, lrate, reg, "MRR", mrr)
            sys.stdout.flush()

###проверка работы авторского решения

In [77]:
create_csr(train, user_item, 'another_train.mtx')
create_csr(test, user_item, 'another_test.mtx')

###BPR_MF Подбор параметров

In [ ]:
list_reg = [0.01, 0.05, 0.1, 0.2]
list_lrate = [0.0001, 0.001, 0.005]
list_K = [10, 15,  20]

bpr_mf_validation = np.zeros((len(list_K), len(list_lrate), len(list_reg)))

for i, K in enumerate(list_K):
    for j, lrate in enumerate(list_lrate):
        for m, reg in enumerate(list_reg):
            cf = rankingmethod.BPR_MF(user_item,K=K, regU=reg, regIpos=reg, regIneg=reg, 
                                      lrate=lrate, verbose=0, maxiter=30)
            cf.fit(train)
            res = rankingmeasure.get_AUC(train, test, 
            cf, skip_train=False)
            bpr_mf_validation[i,j,m] = res
            print(K, lrate, reg, "AUC", res)
            sys.stdout.flush()

In [ ]:
with open('bpr_mf_validation.pickle', 'wb') as f:
    f.dump(bpr_mf_validation, f)

###iMF Подбор  параметров

In [ ]:
list_alpha = [10, 20, 40, 80]
list_lmbd = [0.001, 0.01, 0.05, 0.1]
list_K = [10, 15,  20]

imf_validation = np.zeros((len(list_K), len(list_lrate), len(list_reg)))

for i, K in enumerate(list_K):
    for j, lrate in enumerate(list_lrate):
        for m, reg in enumerate(list_reg):
            cf = rankingmethod.iMF(user_item, K=K, lmbd=0.01, alpha=40, maxiter=30, verbose=2)
            cf.fit(train)
            mrr = rankingmeasure.get_prec_K(train, test, 
            cf, skip_train=False)
            imf_validation[i,j,m] = mrr
            print(K, lrate, reg, "AUC", mrr)
            sys.stdout.flush()

In [ ]:
with open('imf_validation.pickle', 'wb') as f:
    f.dump(bpr_mf_validation, f)

###Проведение такого же эксперимента, как у создатей CLiMF

In [30]:
def climf_experiment(data, list_cf, list_measure, K, maxiter, top):
    res = np.zeros((len(list_measure), len(list_cf)))
    for t in range(maxiter):
        train, test, traintest = givenK_train_test(epinion, K)
        for i, cf in enumerate(list_cf):
            cf.fit(train)
            for j, measure in enumerate(list_measure):
                res[j, i] += measure(train, test, cf)
            print(t, i, res[:, i] / (t + 1))
            sys.stdout.flush()
    res /= K
    return res

In [31]:
list_cf = [
    rankingmethod.CLiMF(user_item, K=10, reg=0.001, lrate=0.01, verbose=0, maxiter=10),
    rankingmethod.BPR_MF(user_item, K = 20, lrate=0.001, regIpos=0.1, 
                              regIneg=0.1, regU=0.1, maxiter=30, verbose=0 ),
    rankingmethod.iMF(user_item, K=20, lmbd=0.01, alpha=40, maxiter=30, verbose=0)
]

list_measure = [
    rankingmeasure.get_prec_K,
    rankingmeasure.get_one_recal_K,
    rankingmeasure.get_MRR,
    rankingmeasure.get_AUC,
    rankingmeasure.get_NDCG
]

In [ ]:
res5 = climf_experiment(epinion, list_cf, list_measure, 5, 5, None)

In [ ]:
res5top = climf_experiment(epinion, list_cf, list_measure, 5, 5, top)

##старые эксперименты

In [70]:
tmp = mmread("../datasets/epinions_original/EP25_UPL5_test.mtx")
test = np.array(list(zip(tmp.row, tmp.col)))
tmp = mmread("../datasets/epinions_original/EP25_UPL5_train.mtx")
train = np.array(list(zip(tmp.row, tmp.col)))
user_item = [max(np.max(test[:,0]), np.max(train[:, 0])) + 1, max(np.max(test[:,1]), np.max(train[:,1])) + 1]

In [71]:
train = create_listarray(train)
test  = create_listarray(test)

In [ ]:
create_csr(train, user_item, '../pyCLiMF-master/good_train.pickle')
create_csr(test, user_item, '../pyCLiMF-master/good_test.pickle')

In [ ]:
randomrec = rankingmethod.RandomRec(user_item)
randomrec.fit(train)
all_measures(train, test, randomrec, False)

In [115]:
poprec = rankingmethod.PopRec(user_item)
poprec.fit(train)
top = poprec.get_list(0)[:3]
# all_measures(train, test, poprec, True, top)
# all_measures(train, test, poprec, True)

In [ ]:
rankingmeasure.get_AUC(train, test, poprec, True, K=user_item[1])

In [ ]:
rankingmeasure.get_NDCG(train, test, poprec, True, K = 5)

In [ ]:
rankingmeasure.get_NDCG(train, test, poprec, True, K = 3)

In [ ]:
index_user = np.random.choice(len(train), 100, replace=False)


In [ ]:
orig_train = create_original_sample(train, index_user)
orig_test = create_original_sample(test, index_user)

In [13]:
climf = rankingmethod.CLiMF(user_item, lrate = 0.0001, reg=0.001, maxiter=15, verbose=2)
climf.fit(train)

iteration 0 loss -98107.253456
iteration 1 loss -98106.9430412
iteration 2 loss -98106.6324012
iteration 3 loss -98106.3214964
iteration 4 loss -98106.0102875
iteration 5 loss -98105.698735
iteration 6 loss -98105.3867996
iteration 7 loss -98105.0744423
iteration 8 loss -98104.7616239
iteration 9 loss -98104.4483056
iteration 10 loss -98104.1344485
iteration 11 loss -98103.8200138
iteration 12 loss -98103.504963
iteration 13 loss -98103.1892575
iteration 14 loss -98102.872859


In [14]:
rankingmeasure.get_MRR(train, test, climf, True)

0.4036119383538297

In [119]:
all_measures(train, test, climf, True, top)

precision5 0.054217888936
1-call5 0.2337855023314964
MRR 0.1290192881946351


KeyboardInterrupt: 

In [117]:
all_measures(train, test, climf, True)

precision5 0.168334039847
1-call5 0.5370919881305638
MRR 0.4057593335701604
AUC 0.346615797654
NGDC 0.190403723158


In [ ]:
climf = rankingmethod.CLiMF(user_item, maxiter=40, lrate = 0.1, verbose=2)
climf.fit(train)


In [ ]:
all_measures(train, test, climf, False)

In [ ]:
all_measures(train, test, climf, True, top)

In [ ]:
list_reg = [0.001, 0.002, 0.005, 0.0005]
list_lrate = [0.05, 0.07, 0.1, 0.12, 0.15 ]
list_K = [10, 12, 15, 7, 5]
times_count = 3

for K, lrate, reg in itertools.product(list_K, list_lrate, list_reg):
    MRR = 0
    
    for t in range(times_count):
        climf = rankingmethod.CLiMF(user_item,K=K, reg=reg, lrate=lrate, verbose=0, maxiter=50)
        climf.fit(train)
        MRR += rankingmeasure.get_MRR(train, test, climf, skip_train=False)
    print("K", K, "lrate", lrate, "reg", reg, "MRR", MRR/times_count)
    sys.stdout.flush()

In [ ]:
bpr_mf = rankingmethod.BPR_MF(user_item, K = 20, lrate=0.001, regIpos=0.1, 
                              regIneg=0.1, regU=0.1, maxiter=30, verbose=2 )

In [ ]:
bpr_mf.fit(train)

In [ ]:
all_measures(train, test, bpr_mf, True)

In [ ]:
rankingmeasure.get_AUC(train, test, bpr_mf, True, K=user_item[1])

In [ ]:
rankingmeasure.get_NDCG(train, test, bpr_mf, True, K=5)

In [ ]:
imf = rankingmethod.iMF(user_item, K=20, lmbd=0.01, alpha=40, maxiter=30, verbose=2)

In [ ]:
imf.fit(train)

In [ ]:
all_measures(train, test, imf, True)

In [ ]:
all_measures(train, test, imf, True, top)

In [ ]:
all_measures(train, test, bpr_mf, True, top)

In [ ]:
all_measures(train, test, poprec, True)

In [ ]:
all_measures(train, test, poprec, True, top)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
a = np.array([False,False, True, True])
b = np.array([0,2,1,3])
print(roc_auc_score(a, b, average=None))

In [ ]:
(a == True).any()

In [ ]:
import random

In [ ]:
random.sample(range(10), 3)